In [1]:
#pragma cling add_include_path("/home/juan/champ_ws/src/champ/champ_base/include/libchamp/include")
#pragma cling add_include_path("/home/juan/champ_ws/src/champ/champ_config/include")
#pragma cling add_include_path("/usr/include/eigen3")

In [2]:
#include "ros/ros.h"
#pragma cling load("roscpp")

#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"
#include "geometry/geometry.h"
#include "robotviz/robotviz.h"

#include "ros/ros.h"
#include "sensor_msgs/JointState.h"
#include "quadruped_base/quadruped_base.h"

#include "quadruped_base/quadruped_components.h"
#include "body_controller/body_controller.h"
#include "leg_controller/leg_controller.h"
#include "kinematics/kinematics.h"

#include "quadruped_description.h"

#include "unistd.h"
#include "iostream"

//#define USE_ROS

In [3]:
ros::Publisher joint_states_publisher;   

#ifdef USE_ROS
int argc = 1;
char arg0[] = "talker";
char* argv[] = {arg0, NULL};
ros::init(argc, argv, "champ_notebook_controller");

ros::NodeHandle nh("");
joint_states_publisher = nh.advertise<sensor_msgs::JointState>("/champ/joint_states", 100);
#endif

std::vector<std::string> joint_names;
joint_names.push_back("lf_hip_joint");
joint_names.push_back("lf_upper_leg_joint");
joint_names.push_back("lf_lower_leg_joint");
joint_names.push_back("rf_hip_joint");
joint_names.push_back("rf_upper_leg_joint");
joint_names.push_back("rf_lower_leg_joint");
joint_names.push_back("lh_hip_joint");
joint_names.push_back("lh_upper_leg_joint");
joint_names.push_back("lh_lower_leg_joint");
joint_names.push_back("rh_hip_joint");
joint_names.push_back("rh_upper_leg_joint");
joint_names.push_back("rh_lower_leg_joint");

In [4]:
champ::GaitConfig gait_config;

gait_config.pantograph_leg = false;
gait_config.max_linear_velocity_x = 0.5;
gait_config.max_linear_velocity_y = 0.5;
gait_config.max_angular_velocity_z = 1.0;
gait_config.swing_height = 0.04;
gait_config.stance_depth = 0.0;
gait_config.stance_duration = 0.25;
gait_config.nominal_height = 0.2;
gait_config.knee_orientation = ">>";

champ::QuadrupedBase base(gait_config);
champ::URDF::loadFromHeader(base);

champ::BodyController body_controller(base);
champ::LegController leg_controller(base);
champ::Kinematics kinematics(base);

In [5]:
xpl::figure fig;

xpl::linear_scale sx, sy;
sx.min = -0.3;
sx.max = 0.3;
sy.min = -0.3;
sy.max = 0.3;

xpl::lines line(sx, sy);
RobotViz robotviz(line);
fig.add_mark(line);

In [6]:
line.colors = std::vector<std::string>({"green"});

In [7]:
xpl::axis hx(sx ), hy(sy);
hy.orientation = "vertical";
fig.add_axis(hx);
fig.add_axis(hy);

In [8]:
void publishJoints(float joint_states[12])
{
    sensor_msgs::JointState joints_msg;

    joints_msg.header.stamp = ros::Time::now();
    joints_msg.name.resize(joint_names.size());
    joints_msg.position.resize(joint_names.size());
    joints_msg.name = joint_names;

    for (size_t i = 0; i < joint_names.size(); ++i)
    {    
        joints_msg.position[i]= joint_states[i];
    }
    #ifdef USE_ROS
    joint_states_publisher.publish(joints_msg);
    #endif
}

In [9]:
champ::Pose req_pose;
req_pose.position.z = gait_config.nominal_height;

champ::Velocities req_vel;
req_vel.linear.x = 0.5;

geometry::Transformation target_foot_positions[4];
float target_joint_positions[12];

fig

A Jupyter widget

In [10]:
for(size_t i = 0;  i < 500; i++)
{    
    body_controller.poseCommand(target_foot_positions, req_pose);
    leg_controller.velocityCommand(target_foot_positions, req_vel);
    kinematics.inverse(target_joint_positions, target_foot_positions);
    
    robotviz.updateJoints(target_joint_positions[1], target_joint_positions[2]);
    //robotviz.updateJoints(-0.75, 0.75);
    
    #ifdef USE_ROS
    publishJoints(target_joint_positions);
    #endif
    
    usleep(10000);
}